In [18]:
import pandas as pd
import json
import numpy as np

In [19]:
DATA_DIR = "/dgx1data/aii/tao/m338824/ADRD/SNOMED_EL"
df = pd.read_json(f"{DATA_DIR}/snomed_predictions_detailed_mixed.jsonl", lines=True)
df1 = df[["concept_id","concept_name","hierarchy","source","gold_concept_id","gold_concept_name","gold_hierarchy","retrieval_eval"]] #

In [20]:
df1

,concept_id,concept_name,hierarchy,source,gold_concept_id,gold_concept_name,gold_hierarchy,retrieval_eval
0,38880002.0,Rigor,finding,sapbert_then_llm_verify,38880002,Rigor (finding),finding,"{'lexical': {'recall@1': 1, 'recall@5': 1, 're..."
1,54706004.0,Mean corpuscular hemoglobin determination,procedure,sapbert_then_llm_verify,54706004,Mean corpuscular hemoglobin determination (pro...,procedure,"{'lexical': {'recall@1': 0, 'recall@5': 0, 're..."
2,126885006.0,Neoplasm of bladder,disorder,sapbert_then_llm_verify,126885006,Neoplasm of bladder (disorder),disorder,"{'lexical': {'recall@1': 0, 'recall@5': 0, 're..."
3,733657002.0,Diverticulosis of colon,finding,llm_selector,733657002,Diverticulosis of colon (finding),finding,"{'lexical': {'recall@1': 0, 'recall@5': 0, 're..."
4,395321006.0,Foley catheter long term use,procedure,fallback_fusion_at_1,439053001,Urinary catheter in situ (finding),finding,"{'lexical': {'recall@1': 0, 'recall@5': 0, 're..."
...,...,...,...,...,...,...,...,...
253,282145008.0,Unable to walk,finding,sapbert_then_llm_verify,282145008,Unable to walk (finding),finding,"{'lexical': {'recall@1': 1, 'recall@5': 1, 're..."
254,442672001.0,Swelling,morphologic abnormality,sapbert_then_llm_verify,442672001,Swelling (morphologic abnormality),morphologic abnormality,"{'lexical': {'recall@1': 1, 'recall@5': 1, 're..."
255,118622000.0,Fistula,morphologic abnormality,sapbert_then_llm_verify,428794004,Fistula (disorder),disorder,"{'lexical': {'recall@1': 0, 'recall@5': 1, 're..."
256,247348008.0,Tenderness,finding,sapbert_then_llm_verify,247348008,Tenderness (finding),finding,"{'lexical': {'recall@1': 1, 'recall@5': 1, 're..."


In [21]:
df1.gold_hierarchy.value_counts()

gold_hierarchy
finding                    94
procedure                  81
disorder                   47
body structure             20
morphologic abnormality    12
regime/therapy              3
cell structure              1
Name: count, dtype: int64

In [22]:
df1 = df1.dropna()
correct = df1[df1["concept_id"].astype(int).astype(str)==df1["gold_concept_id"].astype(str)]

In [23]:
df1.shape, correct.shape

((257, 8), (150, 8))

In [24]:
incorrect = df1[df1["concept_id"].astype(int).astype(str)!=df1["gold_concept_id"].astype(str)]

In [25]:
incorrect.source.value_counts()

source
llm_selector                         43
sapbert_then_llm_verify              22
entitychecker_then_sapbert_verify    19
fallback_fusion_at_1                 12
fallback_sapbert_at_1                11
Name: count, dtype: int64

In [26]:
retrievals = correct["retrieval_eval"].to_list()
results={"lexical":{1:[],5:[],10:[]},"bm25":{1:[],5:[],10:[]},"sapbert_faiss":{1:[],5:[],10:[]},"fusion":{1:[],5:[],10:[]}}
for entry in retrievals:
    for ret_type in entry:
        for rec in entry[ret_type]:
            results[ret_type][int(rec.split("@")[1])].append(entry[ret_type][rec])

In [27]:
for ret_type in results:
    for rec in [1,5,10]:
        print(ret_type, "Recall@",rec,": ",np.mean(results[ret_type][rec]))

lexical Recall@ 1 :  0.5866666666666667
lexical Recall@ 5 :  0.6533333333333333
lexical Recall@ 10 :  0.6733333333333333
bm25 Recall@ 1 :  0.5733333333333334
bm25 Recall@ 5 :  0.68
bm25 Recall@ 10 :  0.7266666666666667
sapbert_faiss Recall@ 1 :  0.8866666666666667
sapbert_faiss Recall@ 5 :  0.9266666666666666
sapbert_faiss Recall@ 10 :  0.94
fusion Recall@ 1 :  0.5555555555555556
fusion Recall@ 5 :  0.8333333333333334
fusion Recall@ 10 :  0.9722222222222222


In [ ]:
# With Synonyms

Num entities = 98

lexical Recall@ 1 :  0.6020408163265306
lexical Recall@ 5 :  0.6836734693877551
lexical Recall@ 10 :  0.6836734693877551
bm25 Recall@ 1 :  0.6224489795918368
bm25 Recall@ 5 :  0.7142857142857143
bm25 Recall@ 10 :  0.7551020408163265
sapbert_faiss Recall@ 1 :  0.5816326530612245
sapbert_faiss Recall@ 5 :  0.6938775510204082
sapbert_faiss Recall@ 10 :  0.6938775510204082
fusion Recall@ 1 :  0.7142857142857143
fusion Recall@ 5 :  0.8673469387755102
fusion Recall@ 10 :  0.9081632653061225

In [ ]:
# Without Synonyms

Num entities = 106

lexical Recall@ 1 :  0.5377358490566038
lexical Recall@ 5 :  0.6037735849056604
lexical Recall@ 10 :  0.6037735849056604
bm25 Recall@ 1 :  0.5566037735849056
bm25 Recall@ 5 :  0.6226415094339622
bm25 Recall@ 10 :  0.660377358490566
sapbert_faiss Recall@ 1 :  0.8490566037735849
sapbert_faiss Recall@ 5 :  0.9245283018867925
sapbert_faiss Recall@ 10 :  0.9245283018867925
fusion Recall@ 1 :  0.7075471698113207
fusion Recall@ 5 :  0.8773584905660378
fusion Recall@ 10 :  0.9056603773584906

In [30]:
CACHE_DIR = "/dgx1data/aii/tao/m338824/ADRD/code/normalization_agent/cache"
SNOMED_CONCEPTS_SYN = f"{CACHE_DIR}/snomedEL_with_synonyms_concepts.parquet"
SNOMED_CONCEPTS_NoSyn = f"{CACHE_DIR}/snomedEL_nosyn_concepts.parquet"
concepts_df_syn = pd.read_parquet(SNOMED_CONCEPTS_SYN)
concepts_df_nosyn = pd.read_parquet(SNOMED_CONCEPTS_NoSyn)

In [31]:
concepts_df_syn

,concept_id,concept_name,hierarchy,synonyms,parent_id,parent_name,parent_hierarchy,depth
0,10000006,Radiating chest pain,finding,[],29857009,Chest pain,finding,9
1,10001005,Bacterial sepsis,disorder,"[Bacterial septicaemia, Bacterial septicemia]",91302008,Sepsis,disorder,5
2,10002003,Resection of stomach fundus,procedure,"[Gastric fundectomy, Gastric fundusectomy]",116175006,Proximal subtotal gastrectomy,procedure,11
3,10006000,Repair of retinal detachment with xenon arc ph...,procedure,[Reattachment of retina by xenon arc photocoag...,78917001,Repair of retina for retinal detachment by pho...,procedure,11
4,10007009,Coffin-Siris syndrome,disorder,[],82354003,Multiple system malformation syndrome,disorder,7
...,...,...,...,...,...,...,...,...
218443,9991008,Abdominal colic,finding,"[Colic, Colicky abdominal pain, Intestinal col...",21522001,Abdominal pain,finding,9
218444,9993006,Incision of diaphragm,procedure,[],367408005,Incision of thorax,procedure,8
218445,9994000,Structure of superior articular process of six...,body structure,[Cranial articular process of sixth thoracic v...,20394004,Structure of superior articular process of tho...,body structure,14
218446,9996003,Arthrotomy with drainage of tarsometatarsal joint,procedure,[],708865003,Arthrotomy with drainage of joint of foot,procedure,9


In [32]:
concepts_df_nosyn

,concept_id,concept_name,hierarchy,synonyms,parent_id,parent_name,parent_hierarchy,depth
0,10000006,Radiating chest pain,finding,[],29857009,Chest pain,finding,9
1,10001005,Bacterial sepsis,disorder,[],91302008,Sepsis,disorder,5
2,10002003,Resection of stomach fundus,procedure,[],116175006,Proximal subtotal gastrectomy,procedure,11
3,10006000,Repair of retinal detachment with xenon arc ph...,procedure,[],78917001,Repair of retina for retinal detachment by pho...,procedure,11
4,10007009,Coffin-Siris syndrome,disorder,[],82354003,Multiple system malformation syndrome,disorder,7
...,...,...,...,...,...,...,...,...
218443,9991008,Abdominal colic,finding,[],21522001,Abdominal pain,finding,9
218444,9993006,Incision of diaphragm,procedure,[],367408005,Incision of thorax,procedure,8
218445,9994000,Structure of superior articular process of six...,body structure,[],20394004,Structure of superior articular process of tho...,body structure,14
218446,9996003,Arthrotomy with drainage of tarsometatarsal joint,procedure,[],708865003,Arthrotomy with drainage of joint of foot,procedure,9
